In [1]:
import glob
import numpy as np
import os
import h5py
import time
import datetime
import sys
import matplotlib.pyplot as plt
from scipy.stats import norm, expon, chi2, uniform
from scipy.stats import chisquare
from scipy.optimize import leastsq, curve_fit, least_squares
import getpass
os.system("echo %s| kinit" %getpass.getpass())

········


0

In [2]:
from keras.models import model_from_json
from keras import callbacks
from keras import metrics, losses, optimizers
from keras.models import Model, Sequential
from keras.layers import Dense, Activation, Input, Conv1D, Flatten, Dropout, LeakyReLU, Layer
from keras.constraints import Constraint, max_norm

class WeightClip(Constraint):
    '''Clips the weights incident to each hidden unit to be inside a range                                                                        \
                                                                                                                                                   
    '''
    def __init__(self, c=2):
        self.c = c
    def __call__(self, p):
        return K.clip(p, -self.c, self.c)
    def get_config(self):
        return {'name': self.__class__.__name__,
                'c': self.c}

def MyModel(nInput, latentsize, layers, weight_clipping):
    inputs = Input(shape=(nInput, ))
    dense  = Dense(latentsize, input_shape=(nInput,), activation='sigmoid', W_constraint = WeightClip(weight_clipping))(inputs)
    for l in range(layers-1):
        dense  = Dense(latentsize, input_shape=(latentsize,), activation='sigmoid', W_constraint = WeightClip(weight_clipping))(dense)
    output = Dense(1, input_shape=(latentsize,), activation='linear', W_constraint = WeightClip(weight_clipping))(dense)
    model = Model(inputs=[inputs], outputs=[output])
    return model



Using TensorFlow backend.


In [3]:
def BuildSample_DY(N_Events, INPUT_PATH, seed, nfiles=20):
    #random integer to select Zprime file between n files                                                                                                                                                             
    u = np.arange(nfiles)#np.random.randint(100, size=100)                                                                                                                                                            
    np.random.shuffle(u)

    #BACKGROUND                                                                                                                                                                                                       
    #extract N_Events from files                                                                                                                                                                                      
    toy_label = INPUT_PATH.split("/")[-2]
    print(toy_label)

    HLF = np.array([])

    for u_i in u:
        f = h5py.File(INPUT_PATH+toy_label+str(u_i+1)+".h5")
        keys=f.keys()
        #check whether the file is empty                                                                                                                                                                              
        if len(keys)==0:
            continue
        cols=np.array([])
        for i in range(len(keys)):
            feature = np.array(f.get(keys[i]))
            feature = np.expand_dims(feature, axis=1)
            if i==0:
                cols = feature
            else:
                cols = np.concatenate((cols, feature), axis=1)
        print(cols.shape)
        np.random.shuffle(cols) #don't want to select always the same event first                                                                                                                                     

        if HLF.shape[0]==0:
            HLF=cols
            i=i+1
        else:
            HLF=np.concatenate((HLF, cols), axis=0)
        f.close()
        #print(HLF_REF.shape)                                                                                                                                                                                         
        if HLF.shape[0]>=N_Events:
            HLF=HLF[:N_Events, :]
            break                                                                                                                                                                                               
    print(HLF.shape)
    # feature order: pt1, pt2, eta1, eta2, delta_phi, mass
    return HLF[:, [4, 5, 1, 2, 0, 3]]

def collect_predictions(DIR_IN):
    '''
    collect predictions from file.
    collect seeds from filename.
    collect t values from file.
    '''
    predictions_set = np.array([])
    seeds_set       = np.array([])
    toy_labels_set  = np.array([])
    t_set           = np.array([])
    cnt = 0
    for fileIN in glob.glob("%s/*_predictions.h5" %DIR_IN):
        # collect seed
        seed = fileIN.split('seed', 1)[1]
        seed = int(seed.split('_', 1)[0])
        print(seed)
        seeds_set = np.append(seeds_set, seed)
        
        # collect toy label ID
        toy_label = fileIN.split('_')[-3]
        toy_labels_set = np.append(toy_labels_set, toy_label)
        
        # collect t
        file_t = fileIN.split('seed', 1)[0]
        file_t = file_t+'t.txt'
        f_t    = open(file_t)
        lines  = f_t.readlines()
        if len(lines)==0:
            continue
        t      =  float(lines[0])
        if(np.isnan(np.array([t]))):
            t  = 0
        t_set  = np.append(t_set, t)
        f_t.close()
        
        # collect predictions
        f = h5py.File(fileIN)
        predictions = f.get("predictions")
        target = f.get("target")
        if not predictions:
            print('continue')
            continue
        predictions = np.array(predictions)
        target      = np.array(target)
        target      = np.expand_dims(target, axis=1)
        if not cnt:
            # initialize the array at the first iteration
            print(predictions.shape)
            predictions_set = predictions
        else:
            # just append to tdistributions_check
            predictions_set = np.concatenate((predictions_set, predictions), axis=1)
        print(str(cnt)+': toy '+toy_label+' loaded.')
        cnt += 1

    print('Final predictions array shape')
    print('(predictions, nr toys)')
    print(predictions_set.shape)
    return predictions_set, seeds_set, toy_labels_set, t_set

### Useful training parameters

In [ ]:
INPUT_PATH_BKG = './DiLepton_SM/'
INPUT_PATH_SIG = './BSM_Detection/DiLepton_Zprime300/
nfile_REF = 66
nfile_SIG = 1


output_path='./WhatNN_haslearnt/Zprime300/'
if not os.path.exists(output_path):
    os.makedirs(output_path)


DIR_INPUT ='.../Z_5D_patience10000_EFT06_ref100000_data20053_epochs300000_latent5_layers3_wclip1.9/'

patience = DIR_INPUT.split("patience",1)[1] 
patience = int(patience.split("_",1)[0])

epochs = DIR_INPUT.split("epochs",1)[1] 
epochs = int(epochs.split("_",1)[0])

wclip = DIR_INPUT.split("wclip",1)[1] 
wclip = float(wclip.split("_",1)[0][:-1])

Nsig = 0
if "sig" in DIR_INPUT.split('/')[-2]:
    Nsig = DIR_INPUT.split("sig",1)[1] 
    Nsig = int(Nsig.split("_",1)[0]) 

Nbkg = 0
if "bkg" in DIR_INPUT.split('/')[-2]:
    print(DIR_INPUT.split('/')[-1])
    Nbkg = DIR_INPUT.split("bkg",1)[1] 
    Nbkg = int(Nbkg.split("_",1)[0] )
    


## Building the Signal reference distribution with all the signal events at our disposal

In [ ]:
#build a signal refernce dataset
Nsig_TOT = 80000
seed_TOT = datetime.datetime.now().microsecond+datetime.datetime.now().second+datetime.datetime.now().minute
if not cut and Nsig:
    # Zprime with no cut
    # build signal
    HLF_SIG, target_SIG, sig_target_SIG = BuildSample_DY(Nsig_TOT, INPUT_PATH_SIG, seed_TOT)

## Read the predictions output for a set of trained NN

In [ ]:
#the seeds are stored as well to rebuild the training data (in particular the reference) of each case
predictions_list, seeds_list, toy_labels_list, t_list = collect_predictions(DIR_INPUT)

# Plot the mass reconstruction made by the NN training and compare it to the refernce mass distribution (BKG only) and the target mass distribution (BGK+SIG)

In [ ]:
def plot_whatNNhaslearnt(predictions_list, seeds_list, toy_labels_list, t_list, 
                         Nsig, Nbkg, Nref, INPUT_PATH_BKG, INPUT_PATH_SIG, nfile_REF, nfile_SIG
                        save_plot=False):
    for i in range(seeds_list.shape[0]):
        # reconstruct training data from seed
        N_Sig_p = np.random.poisson(lam=Nsig, size=1)[0]
        N_Bkg_p = np.random.poisson(lam=Nbkg, size=1)[0]

        #BACKGROUND+REFERENCE
        HLF_REF = BuildSample_DY(N_Events=Nref+N_Bkg_p, INPUT_PATH=INPUT_PATH_BKG, seed=int(seeds_list[i]), nfiles=nfile_REF)
        #SIGNAL                                                                                                                                                                                                                
        HLF_SIG = BuildSample_DY(N_Events=N_Sig_p, INPUT_PATH=INPUT_PATH_SIG, seed=int(seeds_list[i]), nfiles=nfile_SIG)
        #TARGETS
        target_REF  = np.zeros(N_ref)
        target_DATA = np.ones(N_Bkg_p+N_Sig_p)
        target      = np.append(target_REF, target_DATA)
        target      = np.expand_dims(target, axis=1)
        feature     = np.concatenate((HLF_REF, HLF_SIG), axis=0)
        sig_target  = np.append(np.zeros(Nref+N_Bkg_p), np.ones(N_Sig_p))

        #selecting ref events for the invariant mass (last column in feature)
        ref_mass_masked  = np.ma.masked_where(target==1, feature[:, -1])
        ref_mass         = np.ma.compressed(ref_mass_masked)
        #selecting data events for the invariant mass
        data_mass_masked = np.ma.masked_where(target==0, feature[:, -1])
        data_mass        = np.ma.compressed(data_mass_masked)
        #selecting ref predictions of the model
        ref_predictions_masked = np.ma.masked_where(target==1, predictions_list[:, i])
        ref_predictions        = np.ma.compressed(ref_predictions_masked)

        fig=plt.figure(figsize=(10, 10))

        
        # weights 
        ref_weights        = np.ones_like(ref_mass)*Nbkg*1./Nref
        retrieved_weights  = np.exp(ref_predictions)*(Nbkg)*1./Nref
        difference_weights = (retrieved_weights - ref_weights)*Nbkg
        sig_weights        = Nsig*np.ones_like(HLF_SIG[:, -1])*1./HLF_SIG.shape[0]
        bkg_weights        = ref_weights
        merged_weights     = np.concatenate((bkg_weights,sig_weights), axis=0) 
        data_weights       = np.ones_like(data_mass)#*(Nbkg_SB+Nsig_SB)*1./Nref_SB
        
        # building the retrieved distribution and compare it to the reference
        plt.subplot(2, 2, 1)
        bins = np.append(np.linspace(0, 400, 8), np.linspace(500, 2000, 3))
        mass_ref_hist = plt.hist(ref_mass, weights=ref_weights, 
                                range=(0, 2000), bins=bins, 
                                alpha=0.5, 
                                label='REF', linewidth=2)
        mass_retrieved_hist = plt.hist(ref_mass, weights=retrieved_weights,
                                        range=(0, 2000), bins=bins,  
                                        label='DATA RECO', linewidth=3, histtype='step')
        
        plt.title('Toy '+toy_labels_list[i]+': t = '+str(t_SB[i]))
        plt.legend()
        plt.yscale('log')
        plt.ylabel('Probability')
        plt.xlabel('Z mass (GeV)')

        # highlight differences between reco and ref computing the difference  
        plt.subplot(2, 2, 2)
        bins = np.append(np.linspace(0, 400, 8), np.linspace(500, 2000, 3))
        plt.hist(ref_mass, weights=difference_weights, 
                 range=(0, 2000), bins=bins,
                label='DATA RECO - BKG', linewidth=3, histtype='step')
        plt.hist(ref_mass, weights=difference_weights, 
                 range=(bins[4], bins[7]), bins=bins[4:8], alpha=0.5, 
                label='(200-400) GeV')
        
        plt.ylabel('Counts')
        plt.xlabel('Z mass (GeV)')
        plt.title('Toy '+toy_labels_SB[i]+': t = '+str(t_SB[i]))
        plt.legend()
        plt.grid()

        # building the retrieved distribution and compare it to the data
        plt.subplot(2, 2, 3)
        bins = np.append(np.linspace(0, 400, 8), np.linspace(500, 2000, 3))

        mass_SIGref_hist = plt.hist(np.concatenate((ref_mass, HLF_SIG[:, -1]), axis=0), weights=merged_weights,
                                    bins=bins, 
                                    alpha=0.5, color='orange',
                                    label='BKG+SIG distribution', linewidth=2)
        mass_data_hist = plt.hist(data_mass, weights=data_weights,  
                                    bins=bins, color='red',
                                    label='DATA', linewidth=2, histtype='step')
        mass_retrieved_hist = plt.hist(ref_mass, weights=retrieved_weights,
                                    bins=bins, 
                                    label='DATA RECO', linewidth=2, histtype='step')
        plt.legend()
        plt.yscale('log')
        plt.ylabel('Probability')
        plt.xlabel('Z mass (GeV)')

        # building the ratio between data abd data reco
        plt.subplot(2, 2, 4)
        mass_data_counts      = mass_data_hist[0]
        mass_ref_counts       = mass_ref_hist[0]
        mass_retrieved_counts = mass_retrieved_hist[0]
        mass_SIGref_counts    = mass_SIGref_hist[0]
        ratio_reco   = np.array([])
        ratio_data   = np.array([])
        ratio_refSIG = np.array([])
        x = np.array([])
        for j in range(len(bins)-1):
            x_j = 0.5*(bins[j+1]+bins[j])
            if mass_ref_counts[j]!=0:
                x = np.append(x, x_j)
                ratio_reco = np.append(ratio_reco, (mass_retrieved_counts[j]/mass_ref_counts[j]))
                ratio_data = np.append(ratio_data, (mass_data_counts[j]/mass_ref_counts[j]))
                ratio_refSIG = np.append(ratio_refSIG, (mass_SIGref_counts[j]/mass_ref_counts[j]))
            else:
                continue

        plt.scatter(x, ratio_data, color='red', label='DATA / REF')
        plt.scatter(x, ratio_reco, color='blue', label='RECO / REF')
        plt.scatter(x, ratio_refSIG, color='orange', label='(BKG+SIG) / REF')
        plt.plot(x, ratio_data, color='red', linewidth=2)
        plt.plot(x, ratio_reco, color='blue', linewidth=3)
        plt.plot(x, ratio_refSIG, color='orange', linewidth=3)
        plt.legend()
        plt.yscale('log')
        plt.ylabel('Probability')
        plt.xlabel('Z mass (GeV)')
        plt.grid()

        plt.subplots_adjust(left=0.125, right=0.9, bottom = 0.25, wspace=0.3, hspace=0.2)
        plt.show()
        if save_plot:
            fig.savefig(output_path+'toy_'+toy_labels_list[i]+'_reco.png')
        plt.close()
        return